In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
transform = transforms.ToTensor()

In [ ]:
train_data = MNIST(root='data', train=True, transform=transform, download=True)
test_data = MNIST(root='data', train=False, transform=transform, download=True)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 7 * 7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
model = SimpleNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # images = images.view(images.size(0), -1).to(device)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item() 

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Avg Loss: {avg_loss:.4f}")    

In [ ]:
model.eval()
correct = 0
total = 0

In [ ]:

with torch.no_grad():
    for images, labels in test_loader:
        # images = images.view(images.size(0), -1).to(device)
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = outputs.argmax(dim=1)

        total += labels.size(0)
        correct += (predictions == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')



In [ ]:
torch.save(model.state_dict(), 'digit_classifier_cnn.pth')
print('Model saved to digit_classifier_cnn.pth')
# import os
# print(os.getcwd())
# from google.colab import files
# files.download("digit_classifier_cnn.pth")
# print('Model downloaded')


In [ ]:
index = 0
image, true_label = test_data[index]

plt.imshow(image.squeeze(), cmap='gray')
plt.title(f'True Label: {true_label}')
plt.axis('off')
plt.show()

image = image.unsqueeze(0).to(device)  # shape: (1, 1, 28, 28)
# image_flat = image.view(1, -1).to(device)

with torch.no_grad():
    output = model(image)
    predicted_label = output.argmax(dim=1).item()


print(f'Index: {index}')
print(f'True Label: {true_label}')
print(f'Predicted Label: {predicted_label}')

In [ ]:
import random
n = 1000
cnt = 0
for _ in range(n):
    index = random.randint(0, len(test_data)-1)
    image, true_label = test_data[index]

    # plt.imshow(image.squeeze(), cmap='gray')
    # plt.title(f'True Label: {true_label}')
    # plt.axis('off')
    # plt.show()

    image = image.unsqueeze(0).to(device)  # shape: (1, 1, 28, 28)

    with torch.no_grad():
        output = model(image)
        predicted_label = output.argmax(dim=1).item()

    if true_label == predicted_label:
        cnt += 1

    # print('---')
    # print(f'Index: {index}')
    # print(f'True Label: {true_label}')
    # print(f'Predicted Label: {predicted_label}')
print(f'Correct Predictions: {cnt} out of {n}')
print(f'Incorrect Predictions: {n - cnt} out of {n}')
print(f'Accuracy over {n} random samples: {100 * cnt / n} %')